In [2]:
# =============================================================
# BRANCH 4 — REAL-TIME GEO-AI RISK ENGINE
# Outputs:
#   risk_map.html
#   risk_scores.csv
#   hazard_report.txt
# =============================================================

import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap

# -------------------------------------------------------------
# 1. Create synthetic multi-hazard data
# -------------------------------------------------------------
cities = [
    "Delhi", "Mumbai", "Kolkata", "Chennai", "Hyderabad",
    "Bengaluru", "Ahmedabad", "Pune", "Jaipur", "Lucknow"
]

lat = [28.70, 19.07, 22.57, 13.08, 17.38,
       12.97, 23.03, 18.52, 26.91, 26.85]

lon = [77.10, 72.87, 88.36, 80.27, 78.48,
       77.59, 72.58, 73.86, 75.79, 80.95]

np.random.seed(42)

data = pd.DataFrame({
    "city": cities,
    "lat": lat,
    "lon": lon,
    "air_pollution": np.random.randint(150, 400, size=len(cities)),
    "water_pollution": np.random.randint(40, 95, size=len(cities)),
    "industrial_impact": np.random.randint(30, 90, size=len(cities)),
    "heat_stress": np.random.uniform(3.0, 9.0, size=len(cities)),
    "traffic_load": np.random.randint(35, 95, size=len(cities)),
    "waste_pressure": np.random.randint(20, 85, size=len(cities))
})

# -------------------------------------------------------------
# 2. Normalization function
# -------------------------------------------------------------
def normalize(series):
    if series.max() == series.min():
        return series * 0
    return (series - series.min()) / (series.max() - series.min())

# -------------------------------------------------------------
# 3. Normalize all indicators
# -------------------------------------------------------------
cols = [
    "air_pollution",
    "water_pollution",
    "industrial_impact",
    "heat_stress",
    "traffic_load",
    "waste_pressure"
]

for c in cols:
    data[c + "_norm"] = normalize(data[c])

# -------------------------------------------------------------
# 4. Weighted fusion risk formula
# -------------------------------------------------------------
weights = {
    "air_pollution_norm": 0.30,
    "water_pollution_norm": 0.20,
    "industrial_impact_norm": 0.20,
    "heat_stress_norm": 0.10,
    "traffic_load_norm": 0.10,
    "waste_pressure_norm": 0.10
}

data["risk_score"] = (
    data["air_pollution_norm"] * weights["air_pollution_norm"] +
    data["water_pollution_norm"] * weights["water_pollution_norm"] +
    data["industrial_impact_norm"] * weights["industrial_impact_norm"] +
    data["heat_stress_norm"] * weights["heat_stress_norm"] +
    data["traffic_load_norm"] * weights["traffic_load_norm"] +
    data["waste_pressure_norm"] * weights["waste_pressure_norm"]
)

# Risk labels
def label_risk(x):
    if x >= 0.75:
        return "Extreme"
    elif x >= 0.55:
        return "High"
    elif x >= 0.35:
        return "Moderate"
    else:
        return "Low"

data["risk_level"] = data["risk_score"].apply(label_risk)

# -------------------------------------------------------------
# 5. Save CSV output
# -------------------------------------------------------------
data.to_csv("risk_scores.csv", index=False)
print("Saved: risk_scores.csv")

# -------------------------------------------------------------
# 6. Create risk heatmap
# -------------------------------------------------------------
m = folium.Map(location=[22.5, 79.0], zoom_start=5, tiles="CartoDB positron")

heat_data = [
    [row["lat"], row["lon"], row["risk_score"]]
    for _, row in data.iterrows()
]

HeatMap(heat_data, radius=30, blur=20, min_opacity=0.4).add_to(m)

# Add markers
for _, row in data.iterrows():
    popup = (
        "City: " + row["city"] + "<br>" +
        "Risk Score: " + str(round(row["risk_score"], 2)) + "<br>" +
        "Level: " + row["risk_level"] + "<br>" +
        "Air Pollution: " + str(row["air_pollution"]) + "<br>" +
        "Water Pollution: " + str(row["water_pollution"]) + "<br>" +
        "Industrial Impact: " + str(row["industrial_impact"])
    )
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=7,
        fill=True,
        fill_color="red",
        color="red",
        popup=popup
    ).add_to(m)

m.save("risk_map.html")
print("Saved: risk_map.html")

# -------------------------------------------------------------
# 7. Write clean text report
# -------------------------------------------------------------
with open("hazard_report.txt", "w") as f:
    f.write("AetherVision Omega - Branch 4 Hazard Report\n")
    f.write("==========================================\n\n")

    for _, row in data.iterrows():
        f.write("City: " + row["city"] + "\n")
        f.write("Risk Score: " + str(round(row["risk_score"], 2)) + "\n")
        f.write("Risk Level: " + row["risk_level"] + "\n")
        f.write("Air Pollution: " + str(row["air_pollution"]) + "\n")
        f.write("Water Pollution: " + str(row["water_pollution"]) + "\n")
        f.write("Industrial Impact: " + str(row["industrial_impact"]) + "\n")
        f.write("Heat Stress: " + str(round(row["heat_stress"], 1)) + "\n")
        f.write("Traffic Load: " + str(row["traffic_load"]) + "\n")
        f.write("Waste Pressure: " + str(row["waste_pressure"]) + "\n")
        f.write("\n")

print("Saved: hazard_report.txt")

Saved: risk_scores.csv
Saved: risk_map.html
Saved: hazard_report.txt


In [3]:
from google.colab import files

files.download("risk_scores.csv")
files.download("risk_map.html")
files.download("hazard_report.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>